# Word Embeddings

Welcome to the second notebook of this assignment! Word embeddings are numerical representations of words. We can represent words as vectors and maintain their meaning. There are various ways of finding vecotrs for words, equivalently, word embeddings. We explore some of these techniques in this notebook.

## Tabel of Contents

- 1- GloVe: Global Vectors for Word Representation
  - 1.1- Exploring word vectors
  - 1.2- Visualizing in 2-D
- 2- Evaluation
  - 2.1- Cosine similarity
  - 2.2- A little test set
- 3- Learn embeddings
  - 3.1- Co-occurences
  - 3-2. SVD
- 4- Sentiment Analysis of MDB Movie Reviews
  - 4.1- Dataset
  - 4.2- Model
  - 4.3- Train
  - 4.4- Results

## 1- GloVe: Global Vectors for Word Representation

### 1.1- Exploring word vectors

What is the number of words in vocabulary? What is the dimensionality of word vectors?

|         | man   | woman | king  | queen | apple | orange |
| ---     | ---   | ---   |   --- | ---   | ---   |  ---   |
| gender  | -0.99 | 1     | -0.95 | 0.97  | u     | 0.01   |
| royalty | 0.01  | 0.02  | 0.93  | 0.95  | -0.01 | 0.00   |
| fruit   | 0.03  | x     | y     | z     | 0.94  | v      |

What would be the values of $x, y, z, u, v$ and why?

man - woman = king - queen

### 1.2- Visualizing in 2-D

tall -> taller -> tallest, big -> bigger -> biggest

## 2- Evaluation

To evaluate the quality of word vectors, we should look at the similarity of word vectors that are semantically similar, for example word *France* is similar to *Italy*, so their word vectors should be similar. One common metric to measure the distance between word vectors is the **cosine similarity**. The cosine similarity between two words $w_x$ and $w_y$ is defined as
$$
cos(w_x, w_y) = \frac{w_x^Tw_y}{||w_x|| ||w_y||}
$$
, where $||.||$ is the $l_2$ norm.

One evaluation approach that was introduced in [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781) is to consider two pairs of words that have the same type of relationship, for example, *big - bigger* and *small - smaller*. Each two pairs can be viewed as a question: ”What is the word that is similar to small in the same sense as bigger is similar to big?”.

Interestingly, these questions can be answered by doing basic algebraic operations on the word vectors. First we compute $$x = w_{bigger}-w_{big}+w_{small}$$, and then search through all possible word vectors to find the closest one to $x$. In this notebook we use cosine similarity to measure closeness. So the problem can be written as $$ w^{*} = \argmax_{w} cos(w, w_{bigger}-w_{big}+w_{small}) $$.

In [ ]:
test_set = [
    ["Paris", "France", "Toronto", "Canada"],
    ["Australia", "dollar", "Japan", "yen"],
    ["Chicago", "Illinois", "Stockton", "California"],
    ["brother", "sister", "father", "mother"],
    ["apparent", "apparently", "rapid", "rapidly"],
    ["possibly", "impossibly", "ethical", "ethically"],
    ["great", "greater", "tough", "tougher"],
    ["easy", "easiest", "lucky", "luckiest"],
    ["think", "thinking", "read", "reading"],
    ["Switzerland", "Swiss", "Turkey", "Turkish"],
    ["walking", "walked", "swimming", "swam"],
    ["mouse", "mice", "dollar", "dollars"],
    ["work", "works", "speak", "speaks"],
]